In [1]:
from keras.layers import LSTM, Dense
from keras.models import Sequential

Using TensorFlow backend.


In [2]:
import pandas as pd
import numpy as np

In [3]:
pos = pd.read_table('dataset/pos.txt.txt', names = ['x'])
neg = pd.read_table('dataset/neg.txt.txt', names = ['x'])
neg['y'] = 0
pos['y'] = 1
data = pos['x']
data = data.append(neg['x'])
y = pos['y']
y = y.append(neg['y'])

In [4]:
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [5]:
words = pd.read_table('dataset/words.txt', names = ['x'])
words = words.astype(str)

In [ ]:
tok = Tokenizer()
tok.fit_on_texts(words['x'])
seq  = tok.texts_to_sequences(data)

In [66]:
import math

In [69]:

clusters = []
max_len = 80
for i in seq:
    i = np.array(i)
    l = math.ceil(len(i) / max_len)
    padded = pad_sequences([i], l * max_len, padding="post")[0]
    padded = padded.reshape((-1, max_len))
    clusters.append(padded)
clusters = np.array(clusters)
clusters
all_y = to_categorical(y, num_classes=2)

In [81]:
v = len(tok.word_index)

In [143]:
model = Sequential()  
model.add(LSTM( 500, return_sequences=True, input_shape=(None, max_len)))   
model.add(LSTM( 500,  input_shape=(None, max_len)))  
model.add(Dense(250, activation = "tanh"))
model.add(Dense(250, activation = "relu"))
model.add(Dense(250, activation = "sigmoid"))
model.add(Dense(2))

model.compile(optimizer='rmsprop', loss='mse', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_61 (LSTM)               (None, None, 500)         1162000   
_________________________________________________________________
lstm_62 (LSTM)               (None, 500)               2002000   
_________________________________________________________________
dense_41 (Dense)             (None, 250)               125250    
_________________________________________________________________
dense_42 (Dense)             (None, 250)               62750     
_________________________________________________________________
dense_43 (Dense)             (None, 250)               62750     
_________________________________________________________________
dense_44 (Dense)             (None, 2)                 502       
Total params: 3,415,252
Trainable params: 3,415,252
Non-trainable params: 0
_________________________________________________________________


In [96]:
def prepare(clusters, all_y):
    all_data = np.array(list(zip(clusters, all_y)))
    np.random.shuffle(all_data)
    r_x, r_y = all_data.T

    px, py = [], []
    for x, y in zip(r_x, r_y):
        x = np.array([[ [*i] for i in x]]) / (v+1)
        y = np.array([[*y]])
        px.append(x)
        py.append(y)
    return px, py


In [126]:
model.save_weights('new_model.hdf')
model.save_weights('new_model.hdf')

In [147]:
num_epochs = 2000

for i in range(num_epochs):
    
    lc, ac  = 0, 0
    for _x, _y in zip(tx[:100], ty):
        l, a = model.evaluate(_x, _y, verbose=0)
        model.fit(_x, _y, verbose=0, epochs=2)
        if l > a:
            lc += 1
        else:
            ac += 1
        print('Iter : {:10d} accuracy : {:.6f} loss : {:.6f}'.format(ac + lc, a, l), end = '\r')
    acc = ac / (ac + lc)
    print('epoch : {:10d} accuracy : {:.6f} loss : {:.6f}'.format(i, acc, 1 - acc))
    if i % 10 == 0:
        model.save_weights('new_model.hdf')
        tx, ty = prepare(clusters, all_y)
   
        
    

epoch :          0 accuracy : 0.480000 loss : 0.520000
epoch :          1 accuracy : 0.480000 loss : 0.520000
epoch :          2 accuracy : 0.610000 loss : 0.390000
epoch :          3 accuracy : 0.810000 loss : 0.190000
epoch :          4 accuracy : 0.870000 loss : 0.130000
epoch :          5 accuracy : 0.900000 loss : 0.100000


KeyboardInterrupt: 

In [148]:
m = Sequential()  
m.add(LSTM( 500, return_sequences=True, input_shape=(None, max_len))) 
model.compile(optimizer='rmsprop', loss='mse', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_61 (LSTM)               (None, None, 500)         1162000   
_________________________________________________________________
lstm_62 (LSTM)               (None, 500)               2002000   
_________________________________________________________________
dense_41 (Dense)             (None, 250)               125250    
_________________________________________________________________
dense_42 (Dense)             (None, 250)               62750     
_________________________________________________________________
dense_43 (Dense)             (None, 250)               62750     
_________________________________________________________________
dense_44 (Dense)             (None, 2)                 502       
Total params: 3,415,252
Trainable params: 3,415,252
Non-trainable params: 0
_________________________________________________________________
